In [1]:
# generate causal set, estimate coords, compare to original
import dagology as dag
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import example_utils as u
import pickle

In [ ]:
# generate causal set
N = 1000
D = 2
A, R = dag.minkowski_causet(N, D)
R -= 0.5
#A_tr = dag.transitive_reduction(A)

In [ ]:
LP = dag.longest_path_matrix(A)
SP = dag.naive_spacelike_matrix(LP)
X = dag.MDS(SP, D, method='lorentzian')
X = u.flip_time_axis(X, A)
if D==2:
    X = u.flip_space_axis(X, R)
X = u.normalise_coords(X)

In [ ]:
# Compare with a random DAG
A_random = dag.random_dag(N, 16)
LP_random = dag.longest_path_matrix(A_random)
SP_random = dag.naive_spacelike_matrix(LP_random)
X_random = dag.MDS(SP_random, D, method='lorentzian')
X_random = u.flip_time_axis(X_random, A_random)
u.measure_embedding(X_random, A_random, c=1)

In [2]:
def load_causal_set(N, D, i):
    path = '../data/causal_sets/N%s_D%s_%s' % (N, D, i)
    with open(path + '_R.pkl', 'rb') as f:
        R = pickle.load(f)
    with open(path + '_A.pkl', 'rb') as f:
        A = pickle.load(f)   
    with open(path + '_X.pkl', 'rb') as f:
        X = pickle.load(f) 
    return A, R, X

def load_random_dag(N, k, i):
    path = '../data/random_dag/N%s_k%s_%s' % (N, k, i)
    with open(path + '_R.pkl', 'rb') as f:
        R = pickle.load(f)
    with open(path + '_A.pkl', 'rb') as f:
        A = pickle.load(f)   
    return A, R  

In [38]:
A_C2, R_C2, X_C2 = load_causal_set(1000, 2, 0)

In [46]:
A_C3, R_C3, X_C3 = load_causal_set(1000, 3, 0)

In [49]:
A_C4, R_C4, X_C4 = load_causal_set(1000, 4, 0)

In [40]:
A_R, X_R,  = load_random_dag(1000, 16, 0)

In [5]:
def draw_C(A, R, subset=20):
    N, D = R.shape
    assert N == A.shape[0]
    A_sub = A[:subset, :subset]
    A_sub = dag.transitive_reduction(A_sub)
    plt.subplot(1, 1, 1)
    for i in range(min(N, subset)):
        plt.scatter(R[i, 1], R[i, 0], c=u.tableau20[i%20], s=80)
        for j in range(N):
            if i>j and (A_sub[i,j] == 1 or A_sub[j,i]==1):
                plt.plot([R[i,1], R[j,1]], [R[i,0], R[j,0]], 'k-', alpha=0.3)
    #plt.axis('off') 
    plt.show()

In [6]:
def draw_C_embedded(A, R, X=None, subset=20):
    N, D = R.shape
    assert N == A.shape[0]
    A_sub = A[:subset, :subset]
    A_sub = dag.transitive_reduction(A_sub)
    if X is not None:
        assert N == X.shape[0]
        plt.subplot(1, 2, 1)
    for i in range(min(N, subset)):
        plt.scatter(R[i, 1], R[i, 0], c=u.tableau20[i%20], s=80)
        for j in range(N):
            if i>j and (A_sub[i,j] == 1 or A_sub[j,i]==1):
                plt.plot([R[i,1], R[j,1]], [R[i,0], R[j,0]], 'k-', alpha=0.3)
    if X is not None:            
        plt.subplot(1, 2, 2)
        for i in range(min(N, subset)):
            plt.scatter(X[i, 1], X[i, 0], c=u.tableau20[i%20], s=80)
            for j in range(N):
                if i>j and (A_sub[i,j] == 1 or A_sub[j,i]==1):
                    plt.plot([X[i,1], X[j,1]], [X[i,0], X[j,0]], 'k-', alpha=0.3)
                
    plt.show()    

In [ ]:
draw_C_embedded(A, R, X)

In [ ]:
u.average_deviation(R, X)

In [ ]:
u.measure_embedding(X_, A_, c=1)

In [36]:
def create_ROC_data(X, A):
    c_range = np.logspace(-2., 2., 40)
    x, y = [0.], [0.]
    for c in c_range:
        sens, spec = u.receiver_operator_curve(X, A, c)
        x.append(1.-spec)
        y.append(sens)
    x.append(1.)
    y.append(1.)
    return x, y

In [50]:
x_c2, y_c2 = create_ROC_data(X_C2, A_C2)
x_c3, y_c3 = create_ROC_data(X_C3[:,:2], A_C3)
x_c4, y_c4 = create_ROC_data(X_C4[:,:2], A_C4)
x_r, y_r = create_ROC_data(X_R[:,:2], A_R)

In [57]:
plt.plot(x_r, y_r, c=u.tableau20[2], marker='^', linewidth=3, label='Random DAG')
plt.plot(x_c2, y_c2, c=u.tableau20[0], marker='o', linewidth=3, label='2D Causal Set')
plt.plot(x_c3, y_c3, c=u.tableau20[4], marker='v', linewidth=3, label='3D Causal Set')
plt.plot(x_c4, y_c4, c=u.tableau20[6], marker='p', linewidth=3, label='4D Causal Set')
plt.legend(loc='lower right')
plt.show()